In [2]:
import pandas as pd

# Replace 'your_file.tsv' with the path to your TSV file
df = pd.read_csv('mgsm_en.tsv', sep='\t', header=None)

# Display the first few rows of the dataframe
df.head()

,0,1
0,Janet’s ducks lay 16 eggs per day. She eats th...,18
1,A robe takes 2 bolts of blue fiber and half th...,3
2,Josh decides to try flipping a house. He buys...,70000
3,James decides to run 3 sprints 3 times a week....,540
4,"Every day, Wendi feeds each of her chickens th...",20


In [6]:
from openai import OpenAI, RateLimitError
import backoff

client = OpenAI()

@backoff.on_exception(backoff.expo, RateLimitError, max_time=60)
def translate_text(text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You will be provided with a sentence in English, and your task is to translate it into Mongolian. Only output the translation. Do NOT add explanations.",
            },
            {
                "role": "user",
                "content": f"{text}.",
            },
        ],
    )
    return response.choices[0].message.content.strip()

In [7]:
# Apply the translation function to each row in the dataframe
df['translated'] = df[0].apply(lambda x: translate_text(x))

# Display the first few rows of the dataframe with translations
df.head()

,0,1,translated
0,Janet’s ducks lay 16 eggs per day. She eats th...,18,Жанетийн нугаснууд өдөрт 16 өндөг гаргадаг. Тэ...
1,A robe takes 2 bolts of blue fiber and half th...,3,3 боолт.
2,Josh decides to try flipping a house. He buys...,70000,Жош нэг байшинг эргүүлж зарах гэж шийджээ. Тэр...
3,James decides to run 3 sprints 3 times a week....,540,Жеймс долоо хоногт 3 удаа 3 спринт гүйхээр ший...
4,"Every day, Wendi feeds each of her chickens th...",20,40 аяга.


In [9]:
df = df.reindex(columns=['translated', 1, 0])
df.head()

,translated,1,0
0,Жанетийн нугаснууд өдөрт 16 өндөг гаргадаг. Тэ...,18,Janet’s ducks lay 16 eggs per day. She eats th...
1,3 боолт.,3,A robe takes 2 bolts of blue fiber and half th...
2,Жош нэг байшинг эргүүлж зарах гэж шийджээ. Тэр...,70000,Josh decides to try flipping a house. He buys...
3,Жеймс долоо хоногт 3 удаа 3 спринт гүйхээр ший...,540,James decides to run 3 sprints 3 times a week....
4,40 аяга.,20,"Every day, Wendi feeds each of her chickens th..."


In [10]:
df.to_csv('translated_output.tsv', sep='\t', index=False)